In [1]:
import mlflow
mlflow.set_tracking_uri("http://127.0.0.1:5000")

/home/tensor/anaconda3/envs/tensor_check/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [2]:
import tensorflow as tf
from ray import train, tune
from ray.tune.schedulers import ASHAScheduler
from ray.train import RunConfig
from ray.air.integrations.keras import ReportCheckpointCallback
from ray.air.integrations.mlflow import MLflowLoggerCallback
from joblib import load
import numpy as np

2025-01-09 21:02:09.272895: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-09 21:02:09.283978: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-09 21:02:09.298284: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-09 21:02:09.298318: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-09 21:02:09.308197: I tensorflow/core/platform/cpu_feature_gua

In [3]:
def load_dataset():
    import pandas as pd
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import StandardScaler, MinMaxScaler
    
    # Load the CSV file
    # df = pd.read_csv("/mnt/e/Python projects/Movie Recomender/normalized_user_movie_data.csv")

    item_train=pd.read_csv("/mnt/e/Python projects/Movie Recomender/data/content_item_train.csv")
    user_train=pd.read_csv("/mnt/e/Python projects/Movie Recomender/data/content_user_train.csv")
    y_train=pd.read_csv("/mnt/e/Python projects/Movie Recomender/data/content_y_train.csv")
    
    scalerItem = StandardScaler()
    scalerItem.fit(item_train)
    item_train = scalerItem.transform(item_train)
    
    scalerUser = StandardScaler()
    scalerUser.fit(user_train)
    user_train = scalerUser.transform(user_train)
    
    scalerTarget = MinMaxScaler((-1, 1))
    scalerTarget.fit(y_train)
    y_train = scalerTarget.transform(y_train)


    item_train, item_test = train_test_split(item_train, train_size=0.80, shuffle=True, random_state=1)
    user_train, user_test = train_test_split(user_train, train_size=0.80, shuffle=True, random_state=1)
    y_train, y_test       = train_test_split(y_train,    train_size=0.80, shuffle=True, random_state=1)
    
    
    
   
    # X=df
    # # UserTrain=df.iloc[:,2:20]
    # # MovieTrain=df.iloc[:,20:39]
    # y=df['rating']
    
       
    # Split the data into train (80%), temp (20%) split
    # X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Split the temp data into validation (10%) and test (10%) sets
    # X_dev, X_test, y_dev, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
    # UserTrain=X_train.iloc[:,2:20]
    # MovieTrain=X_train.iloc[:,20:39]
    # UserDev=X_dev.iloc[:,2:20]
    # MovieDev=X_dev.iloc[:,20:39]
    # UserTest=X_test.iloc[:,2:20]
    # MovieTest=X_test.iloc[:,20:39]
    return [user_train[:,3:],item_train[:,2:],y_train,user_test[:,3:],item_test[:,2:],y_test]
    #Display the sizes of each set
    # print(f"Training set: {UserTrain.info()},{MovieTrain.info()}, {y_train.shape}")
    # print(f"Validation (Dev) set: {UserDev.shape},{MovieDev.shape}, {y_dev.shape}")
    # print(f"Test set: {UserTest.shape},{MovieTest.shape}, {y_test.shape}")
    #print(X_train)
    # UserTest.to_csv("UserTest.csv", index=False)
    # MovieTest.to_csv("MovieTest.csv",index=False)
    # y_test.to_csv("y_test.csv", index=False)


In [4]:
load_dataset()

[array([[-0.75002545, -0.71201645,  0.05438833, ..., -0.63795668,
         -0.65475297, -0.73380805],
        [-0.4964236 , -0.74715933, -0.08397947, ..., -0.55450152,
         -0.46625411, -0.3946172 ],
        [ 0.3489159 , -0.37815909,  0.38434231, ...,  0.0018662 ,
          0.43274043, -0.02312246],
        ...,
        [ 0.02768689,  0.18412699, -0.03076109, ..., -0.10940734,
          0.21524175, -0.2169458 ],
        [ 1.29569614,  1.20327051,  0.79944571, ...,  1.46233149,
          0.91123753,  1.1398176 ],
        [ 0.0107801 ,  0.09626979,  0.29919289, ...,  0.07141217,
          0.31674114,  0.1060931 ]]),
 array([[-0.74706775, -0.71224094, -0.69696856, ..., -0.39919489,
         -0.50822894, -0.65974585],
        [-1.02211109, -0.71224094,  1.43478495, ..., -0.39919489,
         -0.50822894, -0.65974585],
        [ 1.16171783,  1.40401926, -0.69696856, ..., -0.39919489,
          1.96761721,  1.51573519],
        ...,
        [-0.09735632, -0.71224094, -0.69696856, ..., -

In [5]:
from tensorflow.keras.layers import Layer
class L2Normalization(Layer):
    def call(self, inputs):
        return tf.linalg.l2_normalize(inputs, axis=1)



def train_model(config):
    # Build the model based on the Ray Tune configuration
     # Define the input layer
    input1 = tf.keras.Input(shape=(14,))
    input2 = tf.keras.Input(shape=(15,))

    # First dense layer
    x1 = tf.keras.layers.Dense(config['units_u1'], activation="relu", kernel_initializer="he_normal")(input1)

    # Additional dense layers based on the config
    for i in range(2, config['num_layersu'] + 2):
        x1 = tf.keras.layers.Dense(config[f'unitsu_{i}'], activation="relu", kernel_initializer="he_normal")(x1)

    # Output layer
    embedding1 = tf.keras.layers.Dense(config['emb_unit'], activation="relu")(x1)

     # First dense layer
    x2 = tf.keras.layers.Dense(config['units_m1'], activation="relu", kernel_initializer="he_normal")(input2)

    # Additional dense layers based on the config
    for i in range(2, config['num_layersm'] + 2):
        x2 = tf.keras.layers.Dense(config[f'unitsm_{i}'], activation="relu", kernel_initializer="he_normal")(x2)

    # Output layer
    embedding2 = tf.keras.layers.Dense(config['emb_unit'], activation="relu")(x2)


    emb1 = L2Normalization()(embedding1)
    emb2 = L2Normalization()(embedding2)
    #emb1=tf.linalg.l2_normalize(embedding1, axis=1)
    #emb2=tf.linalg.l2_normalize(embedding2, axis=1)

    output = tf.keras.layers.Dot(axes=1)([emb1, emb2])

    # Create the model
    model = tf.keras.Model(inputs=[input1,input2], outputs=output)

    # Compile the model
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=config['learning_rate']),
        loss=tf.keras.losses.mse,
        metrics=['r2_score','mean_squared_error']
    )

    # Load the dataset
    #(train_data, train_labels), (val_data, val_labels) = tf.keras.datasets.mnist.load_data()
    UserTrain,MovieTrain,y_train,UserDev,MovieDev,y_dev=load_dataset()
    # UserTrain = np.array(UserTrain, dtype=np.float32)
    # MovieTrain = np.array(MovieTrain, dtype=np.float32)
    # y_train = np.array(y_train, dtype=np.float32)
    # UserDev = np.array(UserDev, dtype=np.float32)
    # MovieDev = np.array(MovieDev, dtype=np.float32)
    # y_dev = np.array(y_dev, dtype=np.float32)
    #train_data, val_data = train_data / 255.0, val_data / 255.0
    #train_data, val_data = train_data.reshape(-1, 784), val_data.reshape(-1, 784)
    #train_labels, val_labels = train_labels % 3, val_labels % 3  # Reduce classes for example

    # Train the model with Ray Tune's ReportCheckpointCallback
    model.fit(
        [UserTrain,MovieTrain], y_train,
        epochs=20,
        validation_data=([UserDev,MovieDev], y_dev),
        verbose=0,
        callbacks=[ReportCheckpointCallback(metrics={"r2_score": "r2_score","mse":"mean_squared_error"})]
    )


In [21]:



# Define the search space for hyperparameters
search_space = {
    "units_u1": tune.randint(1, 50),
    "units_m1": tune.randint(1, 50),
    "num_layersu": tune.randint(2, 4),
    "num_layersm": tune.randint(2, 4),
    "learning_rate": tune.loguniform(1e-4, 1e-2),
    "emb_unit":tune.randint(16,32)
}
for i in range(2, 5):  # Add units for layers dynamically based on max num_layers
    search_space[f'unitsu_{i}'] = tune.randint(1, 50)
    search_space[f'unitsm_{i}'] = tune.randint(1, 50)

# Set up ASHA Scheduler
scheduler = ASHAScheduler(
    metric="mse",
    mode="min",
    max_t=50,
    grace_period=5,
    reduction_factor=2
)

# Configure the Tuner and resources
tuner = tune.Tuner(
    tune.with_resources(train_model, {"cpu": 10, "gpu": 1}),
    param_space=search_space,
    tune_config=tune.TuneConfig(
        num_samples=20,
        scheduler=scheduler
    ),
    run_config=train.RunConfig(
            name="agah",
            callbacks=[
                MLflowLoggerCallback(
                    tracking_uri="http://127.0.0.1:5000",
                    experiment_name="movie_recommender_7"  # Databricks MLflow experiment name
                )
            ]
        )
)

# Start tuning and get results
analysis = tuner.fit()
#print("Best hyperparameters found were: ", analysis.get_best_result().config)
#print("All trial results as a DataFrame:\n", analysis.get_dataframe())


(raylet) /home/tensor/anaconda3/envs/tensor_check/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
(raylet)   warnings.warn(
(pid=173793) 2025-01-09 20:11:22.039679: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
(pid=173793) 2025-01-09 20:11:22.055937: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=173793) 2025-01-09 20:11:22.074929: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registere

In [7]:
# from ray.tune.search.bohb import TuneBOHB
# from ray.tune.schedulers import HyperBandForBOHB
from ray import tune
from ray.tune.search.bayesopt import BayesOptSearch
from ray.tune.search.hyperopt import HyperOptSearch

# Define the search space for hyperparameters
search_space = {
    "units_u1": tune.randint(1, 50),
    "units_m1": tune.randint(1, 50),
    "num_layersu": tune.randint(2, 4),
    "num_layersm": tune.randint(2, 4),
    "learning_rate": tune.loguniform(1e-4, 1e-2),
    "emb_unit": tune.randint(16, 32)
}
for i in range(2, 5):  # Add units for layers dynamically based on max num_layers
    search_space[f'unitsu_{i}'] = tune.randint(1, 50)
    search_space[f'unitsm_{i}'] = tune.randint(1, 50)

# # Set up the scheduler for BOHB
# scheduler = HyperBandForBOHB(
#     time_attr="training_iteration", 
#     metric="mse", 
#     mode="min",
#     max_t=50,  # Maximum iterations
#     reduction_factor=2  # Successive halving reduction factor
# )

# Set up BOHB search algorithm
# search_algo = TuneBOHB(metric="mse", mode="min")
hyperopt_search = HyperOptSearch(
    metric="mse", mode="min")

# Configure the tuner
tuner = tune.Tuner(
    tune.with_resources(train_model, {"cpu": 10, "gpu": 1}),
    param_space=search_space,
    tune_config=tune.TuneConfig(
        num_samples=20,
        search_alg=hyperopt_search,
    ),
    run_config=train.RunConfig(
        name="agah_bohb",
        callbacks=[
            MLflowLoggerCallback(
                tracking_uri="http://127.0.0.1:5000",
                experiment_name="movie_recommender_bayes"  # MLflow experiment name
            )
        ]
    )
)

# Start tuning and get results
analysis = tuner.fit()


(raylet) /home/tensor/anaconda3/envs/tensor_check/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
(raylet)   warnings.warn(
(pid=202448) 2025-01-09 21:04:38.374624: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
(pid=202448) 2025-01-09 21:04:38.386109: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=202448) 2025-01-09 21:04:38.405174: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registere

In [8]:
!pip install -U hyperopt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 9.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.3/491.3 kB 10.5 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.8/202.8 kB 11.9 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.1.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [8]:
print("Best hyperparameters found were: ", analysis.get_best_result(metric="r2_score", mode="max").config)

Best hyperparameters found were:  {'units_u1': 45, 'units_m1': 24, 'num_layersu': 2, 'num_layersm': 2, 'learning_rate': 0.0003754957799830956, 'emb_unit': 28, 'unitsu_2': 33, 'unitsm_2': 13, 'unitsu_3': 45, 'unitsm_3': 48, 'unitsu_4': 33, 'unitsm_4': 10}
